In [29]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import base64
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from mongo_module import AnimalShelter # Python Module Import


###########################
# Data Manipulation / Model
###########################
username = "aacuser"    # Updated username for Mongo
password = "user"       # Updated password for Mongo
shelter = AnimalShelter(username, password)
# class read method must support return of cursor object
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

PAGE_SIZE = 10   # Variable for pagination
image_filename = 'GraziosoSalvareLogo.png' # Logo Image and code
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    #    html.Div(id='hidden-div', style={'display':'none'}),
    html.Div([
        # Included Code for image in layout including style
        html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'margin-right': '2em', 'height':'100px', 'width':'100px'}),
        html.Div([
            html.H1('Search & Rescue Identifier'),
            html.H3("Facilitated by Dre' Scheetz", style={'fontSize': 16})])
        ],
        style=dict(display='flex')
    ),

    html.Hr(),
    # Included new Div to contain pagination drop down, text, and filter option so the elemants are in-line.
    html.Div([
        html.Div([
            html.H2("""Results per page""",
                    # Formatting for the text
                    style={'margin-right': '2em', 'fontSize': 16})],
        ),
        # Drop down and selectable values
        dcc.Dropdown(
            id='select_page_size',
            options=[
                {'label': '5', 'value': 5},
                {'label': '10', 'value': 10},
                {'label': '15', 'value': 15},
                {'label': '20', 'value': 20},
            ],
            # Default value for results per page.
            value=10,
            style=dict(width='40%', verticalAlign="middle")
        ),
        # Filtering RadioItems for interactivity of the data table.
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water_filter'},
                {'label': 'Mountain Rescue', 'value': 'mountain_filter'},
                {'label': 'Disaster Rescue', 'value': 'disaster_filter'},
                {'label': 'Reset', 'value': 'no_filter'},
            ],
            value='no_filter',
            labelStyle={'display': 'inline-block'}
        )],
        style=dict(display='flex')
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i,  "deletable ": False,  "selectable ": True} for i in df.columns
        ],
# Included filter and sort along with pagination, row_selectable enabled to single
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        page_action='native',
        page_current=0,
        page_size=PAGE_SIZE,
        data=df.to_dict('records'),
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display' : 'flex'},
             children=[
                dcc.Graph(
                     id="pie-chart",
                     className='col s12 m6',
                ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
# Callback for the drop down input is set to options value, outputs to datatable pagesize.
@app.callback(
    Output('datatable-id', 'page_size'),
    [Input('select_page_size', 'value')]
)
def update_pages(page_size):
    return page_size
# Callback for the RadioItems to select filter options
@app.callback([Output('datatable-id','data'),
    Output('datatable-id','columns')],
    [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    if filter_type == 'water_filter':
        # Mongo_Module CRUD method using the given filter parameters.
        df = pd.DataFrame(list(shelter.read({'$and': [{'$or': [
                                                        # $regex is used due to varied inputs for mixed breeds
                                                        {'breed': {'$regex': 'Labrador'}},
                                                        {'breed': {'$regex': 'Newfoundland'}},
                                                        {'breed': 'Chesapeake Bay Retriever'}]
                                                    },
                                                {'sex_upon_outcome': 'Intact Female'},
                                                # $gte and $lte are used for the age range as greather than or
                                                # equal to and less than or equal to
                                                {'age_upon_outcome_in_weeks': {'$gte': 26,'$lte': 156}}
                                            ]
        })))
    elif filter_type == 'mountain_filter':
        df = pd.DataFrame(list(shelter.read({'$and': [{'breed': {'$in': [
                                                        'German Shepherd',
                                                        'Alaskan Malamute',
                                                        'Old English Sheepdog',
                                                        'Rottweiler',
                                                        'Siberian Husky']
                                                        }},
                                                {'sex_upon_outcome': 'Intact Male'},
                                                {'age_upon_outcome_in_weeks': {'$gte': 26,'$lte': 156}}
                                            ]
        })))
    elif filter_type == 'disaster_filter':
        df = pd.DataFrame(list(shelter.read({'$and': [{'breed': {'$in': [
                                                        'Doberman Pinscher',
                                                        'German Shepherd',
                                                        'Golden Retriever',
                                                        'Bloodhound',
                                                        'Rottweiler']
                                                    }},
                                                {'sex_upon_outcome': 'Intact Male'},
                                                {'age_upon_outcome_in_weeks': {'$gte': 20,'$lte': 300}}
                                            ]
        })))
    elif filter_type == 'no_filter':
        df = pd.DataFrame.from_records(shelter.read({}))

    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output("pie-chart", "figure"),
    [Input("datatable-id", "data")]
)
def generate_chart(data):
    dff = pd.DataFrame.from_dict(data) #Assign data from read
    fig = px.pie(
        dff,
        names='breed',
        hole=.3
    )
    return fig

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data'),
    Input('datatable-id', 'selected_rows')]
)
# Function of geolocation chart callback
def update_map(viewData, selected_rows):
    dff = pd.DataFrame.from_dict(viewData)
    # Sets default selected_rows to the first in data table if none is selected.
    if selected_rows is None:
        selected_rows = [0]
    # When row is selected 
    if len(selected_rows) == 1:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[(dff.iloc[selected_rows[0],12]), (dff.iloc[selected_rows[0],13])], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Position of marker is set to the longitude and latitude of the selected row
                dl.Marker(position=[(dff.iloc[selected_rows[0],12]), (dff.iloc[selected_rows[0],13])], children=[
                    dl.Tooltip(dff.iloc[selected_rows[0],4]),
                    dl.Popup([
                        # Popup set to display from the selected row of data table.
                        html.P(["Name: ", dff.iloc[selected_rows[0],8]]),
                        html.P(["Outcome: ", dff.iloc[selected_rows[0],10]]),
                        html.P(["Sex: ", dff.iloc[selected_rows[0],11]]),
                        html.P(["Breed: ", dff.iloc[selected_rows[0],4]]),
                        html.P(["Age: ", dff.iloc[selected_rows[0],14]])
                    ])
                ])
            ])
        ]


app